In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Import

In [2]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.offline as off
import plotly.graph_objs as go
import colorlover as cl

from db.mongo_connection import MyMongo
from util.util import save_df_to_tsv
from my_plotly.draw import (get_horizontal_bar_layout, get_hovertext_list, get_party_color_dict, make_annotation,
get_header_and_cells_from_df, get_legend_rect_party, get_legend_text_party, party_color)

off.init_notebook_mode(connected=True)

# Function

# Mongo Connection

In [3]:
with MyMongo() as db:
    raw_conference = db.get_df_from_table('assembly', 'watch_conference')
    raw_rollbook = db.get_df_from_table('assembly', 'watch_rollbook')
    raw_member = db.get_df_from_table('assembly', 'watch_member')
#     committee = db.get_df_from_table('assembly', 'committee')

raw_conference = raw_conference.drop(columns=['_id'])
raw_rollbook = raw_rollbook.drop(columns=['_id'])
raw_member = raw_member.drop(columns=['_id']); raw_member

<--Mongo Connected.
Mongo Connection Closed.-->


,Email,empNm,hjNm,seq,당선횟수,선거구,소속위원회,연락처,정당,주요경력,학력
0,radiohaha@naver.com,하태경,河泰慶,727,"2선 [ 19대, 20대 ]",부산 해운대구갑,환경노동,의원회관 939호 | 전화 02-784-2491 | 팩스 02-788-0382,바른미래당,새누리당 북한인권 및 탈북자납북자위원회 위원장 | 민주평화통일자문회의 자문위원 | ...,브니엘고등학교 | 서울대학교 물리학과 | 고려대학교국제대학원 석사
1,dreamham7@na.go.kr,함진규,咸珍圭,795,"2선 [ 19대, 20대 ]",경기 시흥시갑,국토교통 | 예산결산특별,의원회관 547호 | 전화 02-784-4277 | 팩스 02-788-0387,자유한국당,새누리당 대변인 | 한나라당 수석부대변인 | 경기도의회 의원,고려대학교대학원 법학 박사 수료
2,7882971@hanmail.net,홍의락,洪宜洛,843,"2선 [ 19대, 20대 ]",대구 북구을,산업통상자원중소벤처기업 | 예산결산특별 | 4차 산업혁명 특별,의원회관 617호 | 전화 02-784-6277 | 팩스 02-788-0392,더불어민주당,민주당 경상북도당위원장 | 열린정책연구원 이사,고려대학교 농업경제학과
3,peace@peacehong.net,홍익표,洪翼杓,708,"2선 [ 19대, 20대 ]",서울 중구성동구갑,산업통상자원중소벤처기업,의원회관 839호 | 전화 02-784-6887 | 팩스 02-788-0393,더불어민주당,민주당 원내대변인 | 인천광역시 남북교류협력위원 | 북한대학원대학교 겸임교수 | 통...,한양대학교대학원 정치학 박사
4,pv21c@chol.com,황주홍,黃柱洪,769,"2선 [ 19대, 20대 ]",전남 고흥군보성군장흥군강진군,농림축산식품해양수산 | 예산결산특별,의원회관 919호 | 전화 02-784-8834 | 팩스 02-788-0399,민주평화당,아태평화재단 부총장 겸 연구실장 | 건국대학교 정치외교학과 교수 | 전라남도 강진군 군수,광주제일고등학교 | 연세대학교 정치외교학과 | 연세대학교대학원 정치학 석사 | 미국...
5,hansunkyo@assembly.go.kr,한선교,韓善敎,475,"4선 [ 17대, 18대, 19대, 20대 ]",경기 용인시병,교육문화체육관광,의원회관 913호 | 전화 02-784-2066 | 팩스 02-788-0385,자유한국당,"한나라당 홍보기획본부장 | 제17,18대 국회의원 | 한나라당 대변인 | MBC 아나운서",대일고등학교 | 성균관대학교 물리학과 | 성균관대학교국가전략대학원 정치학 석사
6,cma2008@naver.com,추미애,秋美愛,256,"5선 [ 15대, 16대, 18대, 19대, 20대 ]",서울 광진구을,외교통일,의원회관 501호 | 전화 02-784-1270 | 팩스 02-788-0381,더불어민주당,더불어민주당 최고위원 | 법무법인아주 대표변호사 | 새천년민주당 최고위원 | 광주고...,경북여자고등학교 | 한양대학교 법학과 | 한양대학교대학원 법학 석사 수료 | 연...
7,youngvote@hanmail.net,홍영표,洪永杓,680,"3선 [ 18대, 19대, 20대 ]",인천 부평구을,환경노동 | 국회운영 | 정보,의원회관 1004호 | 전화 02-784-3143 | 팩스 02-788-0391,더불어민주당,더불어민주당 인천시당 위원장 | 민주당 원내대변인 | 재정경제부 FTA국내대책 본부...,동국대학교 철학과 | 동국대학교대학원 행정학 석사 | 동국대학교대학원 행정학 박사 수료
8,mjhong2004@na.go.kr,홍문종,洪文鐘,266,"4선 [ 15대, 16대, 19대, 20대 ]",경기 의정부시을,외교통일,의원회관 848호 | 전화 02-784-4777 | 팩스 02-788-0389,자유한국당,새누리당 사무총장 | 경민대학교 총장 | 한나라당 경기도당위원장,대광고등학교 | 고려대학교 교육학과 | 고려대학교대학원 교육학 석사 | 하버드대학교...
9,mphonglove@hanmail.net,홍문표,洪文杓,477,"3선 [ 17대, 19대, 20대 ]",충남 홍성군예산군,농림축산식품해양수산,의원회관 336호 | 전화 02-784-9587 | 팩스 02-788-0390,자유한국당,새누리당 제1사무부총장 | 한국농업경영인중앙연합회 명예회원 | 한국농어촌공사 사장,한영고등학교 | 건국대학교 | 한양대학교 대학원 행정학 석사


# conference

In [4]:
print(len(raw_conference))
raw_conference

1290


,committee_id,committee_round,committeename,confer_date,confer_id,rollbook_fetched
0,보건복지위원회,제343회 02차,보건복지위원회,2016-06-21,9002,True
1,환경노동위원회,제343회 01차,환경노동위원회,2016-06-21,9092,True
2,법제사법위원회,제343회 01차,법제사법위원회,2016-06-20,8994,True
3,농림축산식품해양수산위원회,제343회 01차,농림축산식품해양수산위원회,2016-06-20,8995,True
4,산업통상자원중소벤처기업위원회,제343회 01차,산업통상자원중소벤처기업위원회,2016-06-20,8996,True
5,국토교통위원회,제343회 01차,국토교통위원회,2016-06-20,8997,True
6,정보위원회,제343회 01차,정보위원회,2016-06-20,8998,True
7,교육문화체육관광위원회,제343회 01차,교육문화체육관광위원회,2016-06-17,8992,True
8,여성가족위원회,제343회 01차,여성가족위원회,2016-06-17,8993,True
9,보건복지위원회,제343회 01차,보건복지위원회,2016-06-15,8991,True


# confer_committee

In [5]:
confer_committee = raw_conference.groupby(['confer_id', 'committee_id']).agg({'committee_round': 'count'}).reset_index(); confer_committee
print(len(confer_committee))
confer_committee

1290


,confer_id,committee_id,committee_round
0,10000,교육문화체육관광위원회,1
1,10001,법제사법위원회,1
2,10002,정무위원회,1
3,10003,기획재정위원회,1
4,10004,과학기술정보방송통신위원회,1
5,10005,교육문화체육관광위원회,1
6,10006,외교통일위원회,1
7,10007,외교통일위원회,1
8,10008,국방위원회,1
9,10009,행정안전위원회,1


# rollbook

In [6]:
rollbook = raw_rollbook.merge(confer_committee[['confer_id', 'committee_id']], on='confer_id', how='left')
rollbook = rollbook.merge(raw_member[['seq', '당선횟수']], on='seq', how='left')
# rollbook['선수'] = rollbook['당선횟수'].str[0].astype(int)

# Fill NA
nans = {
    '권석창': ['자유한국당', '상실, 선거법위반', '1선'],
    '김경수': ['더불어민주당', '사퇴, 경남도지사후보', '1선'],
    '김종인': ['더불어민주당', '사퇴, 탈당', '5선'],
    '김종태': ['자유한국당', '상실, 선거법위반, 당시 새누리당', '2선'],
    '문미옥': ['더불어민주당', '사퇴, 대통령비서실 과학기술보좌관', '1선'],
    '박남춘': ['더불어민주당', '사퇴, 인천시장후보', '2선'],
    '박준영': ['민주평화당', '상실, 선거법위반', '1선'],
    '박찬우': ['자유한국당', '상실, 선거법위반', '1선'],
    '배덕광': ['자유한국당', '사퇴, 엘시티게이트', '2선'],
    '송기석': ['바른미래당', '상실, 선거법위반', '1선'],
    '안철수': ['바른미래당', '사퇴, 대통령후보', '2선'],
    '양승조': ['더불어민주당', '사퇴, 충남도지사후보', '4선'],
    '윤종오': ['민중당', '상실, 선거법위반, 당시 무소속', '1선'],
    '이철우': ['자유한국당', '사퇴, 경북도지사후보', '3선'],
    '최명길': ['바른미래당', '상실, 선거법위반', '1선'],
}
for key, value in nans.items():
    rollbook.loc[rollbook['empNm']==key, '당선횟수'] = value[2]

# Add column: 선수
rollbook['선수'] = rollbook['당선횟수'].str[:1].astype(int); rollbook

# Remove . from empNm
rollbook.loc[:,['empNm']] = rollbook['empNm'].str.replace('.', '')

print(len(rollbook))
rollbook

28208


,attended,confer_id,empNm,party,seq,committee_id,당선횟수,선수
0,출석,9002,김광수,국민의당,1013,보건복지위원회,1선,1
1,출석,9002,천정배,국민의당,246,보건복지위원회,"6선 [ 15대, 16대, 17대, 18대, 19대, 20대 ]",6
2,출석,9002,최도자,국민의당,960,보건복지위원회,1선,1
3,출석,9002,권미혁,더불어민주당,946,보건복지위원회,1선,1
4,출석,9002,기동민,더불어민주당,885,보건복지위원회,1선,1
5,출석,9002,김상희,더불어민주당,552,보건복지위원회,"3선 [ 18대, 19대, 20대 ]",3
6,출석,9002,남인순,더불어민주당,833,보건복지위원회,"2선 [ 19대, 20대 ]",2
7,출석,9002,양승조,더불어민주당,402,보건복지위원회,4선,4
8,출석,9002,오제세,더불어민주당,406,보건복지위원회,"4선 [ 17대, 18대, 19대, 20대 ]",4
9,출석,9002,인재근,더불어민주당,712,보건복지위원회,"2선 [ 19대, 20대 ]",2


In [7]:
rollbook.loc[rollbook['empNm'].str.find('.')!=-1]

,attended,confer_id,empNm,party,seq,committee_id,당선횟수,선수


In [8]:
rollbook.loc[rollbook['empNm']=='전재수'].groupby('committee_id').agg({'committee_id':'count'})

,committee_id
committee_id,
교육문화체육관광위원회,77
민생경제특별위원회,8
예산결산특별위원회,17
윤리특별위원회,2


## rollbook_standing

In [9]:
rollbook_standing = rollbook.loc[(rollbook['committee_id'].str.find('인사')==-1)&
                                (rollbook['committee_id'].str.find('특별')==-1)]
print(len(rollbook_standing))
rollbook_standing

22411


,attended,confer_id,empNm,party,seq,committee_id,당선횟수,선수
0,출석,9002,김광수,국민의당,1013,보건복지위원회,1선,1
1,출석,9002,천정배,국민의당,246,보건복지위원회,"6선 [ 15대, 16대, 17대, 18대, 19대, 20대 ]",6
2,출석,9002,최도자,국민의당,960,보건복지위원회,1선,1
3,출석,9002,권미혁,더불어민주당,946,보건복지위원회,1선,1
4,출석,9002,기동민,더불어민주당,885,보건복지위원회,1선,1
5,출석,9002,김상희,더불어민주당,552,보건복지위원회,"3선 [ 18대, 19대, 20대 ]",3
6,출석,9002,남인순,더불어민주당,833,보건복지위원회,"2선 [ 19대, 20대 ]",2
7,출석,9002,양승조,더불어민주당,402,보건복지위원회,4선,4
8,출석,9002,오제세,더불어민주당,406,보건복지위원회,"4선 [ 17대, 18대, 19대, 20대 ]",4
9,출석,9002,인재근,더불어민주당,712,보건복지위원회,"2선 [ 19대, 20대 ]",2


In [10]:
# rollbook_standing.loc[(rollbook_standing['seq']=='465')]
# rollbook_standing.loc[(rollbook_standing['committee_id']=='외교통일위원회')&(rollbook_standing['empNm']=='최경환')]
rollbook_standing.loc[(rollbook_standing['empNm']=='김영진')&(rollbook_standing['attended']=='청가')]
# 최경환: 465

,attended,confer_id,empNm,party,seq,committee_id,당선횟수,선수
14925,청가,9910,김영진,더불어민주당,982,행정안전위원회,1선,1


# grb_committee_freq

In [11]:
grb_committee_freq = rollbook.groupby(['committee_id']).agg({'선수': 'mean'}); grb_committee_freq
grb_committee_freq = grb_committee_freq.reset_index(); grb_committee_freq
grb_committee_freq['선수'] = round(grb_committee_freq['선수'], 2); grb_committee_freq

,committee_id,선수
0,4차 산업혁명 특별위원회,1.46
1,가습기살균제 사고 진상규명과 피해구제 및 재발방지 대책마련을 위한 국정조사특별위원회,1.34
2,감사원장(최재형)임명동의에 관한 인사청문특별위원회,1.46
3,과학기술정보방송통신위원회,1.85
4,교육문화체육관광위원회,1.91
5,국무총리(이낙연) 임명동의에 관한 인사청문특별위원회위원회,1.62
6,국방위원회,2.48
7,국토교통위원회,2.16
8,국회운영위원회,1.56
9,기획재정위원회,2.59


# grb_committee

In [12]:
grb_committee = rollbook.groupby(['committee_id', 'attended']).agg({'committee_id': 'count'}); grb_committee
grb_committee = grb_committee.unstack(); grb_committee

grb_committee.columns = grb_committee.columns.droplevel(); grb_committee
grb_committee.columns.name = ''
grb_committee = grb_committee.reset_index(); grb_committee
grb_committee = grb_committee.fillna(0); grb_committee
grb_committee.iloc[:, 1:5] = grb_committee.iloc[:, 1:5].applymap(lambda x: int(x)); grb_committee

grb_committee['합계'] = grb_committee['결석'] + grb_committee['청가'] + grb_committee['출석'] + grb_committee['출장']; grb_committee
grb_committee['결석률'] = round(grb_committee['결석'] / grb_committee['합계'] * 100, 2); grb_committee
grb_committee['출석률'] = round(grb_committee['출석'] / grb_committee['합계'] * 100, 2); grb_committee

grb_committee = grb_committee.merge(grb_committee_freq, on='committee_id', how='left'); grb_committee

,committee_id,결석,청가,출석,출장,합계,결석률,출석률,선수
0,4차 산업혁명 특별위원회,19,18,233,0,270,7.04,86.30,1.46
1,가습기살균제 사고 진상규명과 피해구제 및 재발방지 대책마련을 위한 국정조사특별위원회,40,7,153,0,200,20.00,76.50,1.34
2,감사원장(최재형)임명동의에 관한 인사청문특별위원회,5,1,20,0,26,19.23,76.92,1.46
3,과학기술정보방송통신위원회,143,73,1198,0,1414,10.11,84.72,1.85
4,교육문화체육관광위원회,238,80,1911,0,2229,10.68,85.73,1.91
5,국무총리(이낙연) 임명동의에 관한 인사청문특별위원회위원회,0,1,51,0,52,0.00,98.08,1.62
6,국방위원회,137,28,837,0,1002,13.67,83.53,2.48
7,국토교통위원회,151,60,1629,0,1840,8.21,88.53,2.16
8,국회운영위원회,102,55,846,0,1003,10.17,84.35,1.56
9,기획재정위원회,159,125,1440,0,1724,9.22,83.53,2.59


# grb_member

In [13]:
grb_member = rollbook.groupby(['empNm', 'seq', 'attended']).agg({'seq': 'count'}); grb_member
grb_member = grb_member.unstack(); grb_member
grb_member.columns = grb_member.columns.droplevel(); grb_member
grb_member.columns.name = ''
grb_member = grb_member.reset_index(); grb_member
grb_member = grb_member.fillna(0); grb_member
grb_member.iloc[:, 2:6] = grb_member.iloc[:, 2:6].applymap(lambda x: int(x)); grb_member
grb_member['합계'] = grb_member['결석'] + grb_member['청가'] + grb_member['출석'] + grb_member['출장']; grb_member
grb_member['결석률'] = grb_member['결석'] / grb_member['합계'] * 100; grb_member
grb_member['출석률'] = grb_member['출석'] / grb_member['합계'] * 100; grb_member
grb_member = grb_member.merge(raw_member[['seq', '정당', '당선횟수']], on='seq', how='left'); grb_member

# Fill NA
att_nans = grb_member.loc[grb_member['정당'].isna()]
nans = {
    '권석창': ['자유한국당', '상실, 선거법위반', '1선'],
    '김경수': ['더불어민주당', '사퇴, 경남도지사후보', '1선'],
    '김종인': ['더불어민주당', '사퇴, 탈당', '5선'],
    '김종태': ['자유한국당', '상실, 선거법위반, 당시 새누리당', '2선'],
    '문미옥': ['더불어민주당', '사퇴, 대통령비서실 과학기술보좌관', '1선'],
    '박남춘': ['더불어민주당', '사퇴, 인천시장후보', '2선'],
    '박준영': ['민주평화당', '상실, 선거법위반', '1선'],
    '박찬우': ['자유한국당', '상실, 선거법위반', '1선'],
    '배덕광': ['자유한국당', '사퇴, 엘시티게이트', '2선'],
    '송기석': ['바른미래당', '상실, 선거법위반', '1선'],
    '안철수': ['바른미래당', '사퇴, 대통령후보', '2선'],
    '양승조': ['더불어민주당', '사퇴, 충남도지사후보', '4선'],
    '윤종오': ['민중당', '상실, 선거법위반, 당시 무소속', '1선'],
    '이철우': ['자유한국당', '사퇴, 경북도지사후보', '3선'],
    '최명길': ['바른미래당', '상실, 선거법위반', '1선'],
}
for key, value in nans.items():
    grb_member.loc[grb_member['empNm']==key, '정당'] = value[0]
    grb_member.loc[grb_member['empNm']==key, '특이사항'] = value[1]
    grb_member.loc[grb_member['empNm']==key, '당선횟수'] = value[2]

# Add column: 선수
grb_member['선수'] = grb_member['당선횟수'].str[:1].astype(int); grb_member

,empNm,seq,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항,선수
0,강길부,318,23,5,85,0,113,20.353982,75.221239,무소속,"4선 [ 17대, 18대, 19대, 20대 ]",NaN,4
1,강병원,889,6,6,112,0,124,4.838710,90.322581,더불어민주당,1선,NaN,1
2,강석진,972,13,0,82,0,95,13.684211,86.315789,자유한국당,1선,NaN,1
3,강석호,537,16,0,69,0,85,18.823529,81.176471,자유한국당,"3선 [ 18대, 19대, 20대 ]",NaN,3
4,강창일,320,17,4,73,0,94,18.085106,77.659574,더불어민주당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN,4
5,강효상,935,25,13,65,0,103,24.271845,63.106796,자유한국당,1선,NaN,1
6,강훈식,1005,5,3,74,0,82,6.097561,90.243902,더불어민주당,1선,NaN,1
7,경대수,755,13,0,77,0,90,14.444444,85.555556,자유한국당,"2선 [ 19대, 20대 ]",NaN,2
8,고용진,887,4,5,65,0,74,5.405405,87.837838,더불어민주당,1선,NaN,1
9,곽대훈,911,19,0,78,0,97,19.587629,80.412371,자유한국당,1선,NaN,1


# member

In [14]:
print(len(raw_member))
raw_member

288


,Email,empNm,hjNm,seq,당선횟수,선거구,소속위원회,연락처,정당,주요경력,학력
0,radiohaha@naver.com,하태경,河泰慶,727,"2선 [ 19대, 20대 ]",부산 해운대구갑,환경노동,의원회관 939호 | 전화 02-784-2491 | 팩스 02-788-0382,바른미래당,새누리당 북한인권 및 탈북자납북자위원회 위원장 | 민주평화통일자문회의 자문위원 | ...,브니엘고등학교 | 서울대학교 물리학과 | 고려대학교국제대학원 석사
1,dreamham7@na.go.kr,함진규,咸珍圭,795,"2선 [ 19대, 20대 ]",경기 시흥시갑,국토교통 | 예산결산특별,의원회관 547호 | 전화 02-784-4277 | 팩스 02-788-0387,자유한국당,새누리당 대변인 | 한나라당 수석부대변인 | 경기도의회 의원,고려대학교대학원 법학 박사 수료
2,7882971@hanmail.net,홍의락,洪宜洛,843,"2선 [ 19대, 20대 ]",대구 북구을,산업통상자원중소벤처기업 | 예산결산특별 | 4차 산업혁명 특별,의원회관 617호 | 전화 02-784-6277 | 팩스 02-788-0392,더불어민주당,민주당 경상북도당위원장 | 열린정책연구원 이사,고려대학교 농업경제학과
3,peace@peacehong.net,홍익표,洪翼杓,708,"2선 [ 19대, 20대 ]",서울 중구성동구갑,산업통상자원중소벤처기업,의원회관 839호 | 전화 02-784-6887 | 팩스 02-788-0393,더불어민주당,민주당 원내대변인 | 인천광역시 남북교류협력위원 | 북한대학원대학교 겸임교수 | 통...,한양대학교대학원 정치학 박사
4,pv21c@chol.com,황주홍,黃柱洪,769,"2선 [ 19대, 20대 ]",전남 고흥군보성군장흥군강진군,농림축산식품해양수산 | 예산결산특별,의원회관 919호 | 전화 02-784-8834 | 팩스 02-788-0399,민주평화당,아태평화재단 부총장 겸 연구실장 | 건국대학교 정치외교학과 교수 | 전라남도 강진군 군수,광주제일고등학교 | 연세대학교 정치외교학과 | 연세대학교대학원 정치학 석사 | 미국...
5,hansunkyo@assembly.go.kr,한선교,韓善敎,475,"4선 [ 17대, 18대, 19대, 20대 ]",경기 용인시병,교육문화체육관광,의원회관 913호 | 전화 02-784-2066 | 팩스 02-788-0385,자유한국당,"한나라당 홍보기획본부장 | 제17,18대 국회의원 | 한나라당 대변인 | MBC 아나운서",대일고등학교 | 성균관대학교 물리학과 | 성균관대학교국가전략대학원 정치학 석사
6,cma2008@naver.com,추미애,秋美愛,256,"5선 [ 15대, 16대, 18대, 19대, 20대 ]",서울 광진구을,외교통일,의원회관 501호 | 전화 02-784-1270 | 팩스 02-788-0381,더불어민주당,더불어민주당 최고위원 | 법무법인아주 대표변호사 | 새천년민주당 최고위원 | 광주고...,경북여자고등학교 | 한양대학교 법학과 | 한양대학교대학원 법학 석사 수료 | 연...
7,youngvote@hanmail.net,홍영표,洪永杓,680,"3선 [ 18대, 19대, 20대 ]",인천 부평구을,환경노동 | 국회운영 | 정보,의원회관 1004호 | 전화 02-784-3143 | 팩스 02-788-0391,더불어민주당,더불어민주당 인천시당 위원장 | 민주당 원내대변인 | 재정경제부 FTA국내대책 본부...,동국대학교 철학과 | 동국대학교대학원 행정학 석사 | 동국대학교대학원 행정학 박사 수료
8,mjhong2004@na.go.kr,홍문종,洪文鐘,266,"4선 [ 15대, 16대, 19대, 20대 ]",경기 의정부시을,외교통일,의원회관 848호 | 전화 02-784-4777 | 팩스 02-788-0389,자유한국당,새누리당 사무총장 | 경민대학교 총장 | 한나라당 경기도당위원장,대광고등학교 | 고려대학교 교육학과 | 고려대학교대학원 교육학 석사 | 하버드대학교...
9,mphonglove@hanmail.net,홍문표,洪文杓,477,"3선 [ 17대, 19대, 20대 ]",충남 홍성군예산군,농림축산식품해양수산,의원회관 336호 | 전화 02-784-9587 | 팩스 02-788-0390,자유한국당,새누리당 제1사무부총장 | 한국농업경영인중앙연합회 명예회원 | 한국농어촌공사 사장,한영고등학교 | 건국대학교 | 한양대학교 대학원 행정학 석사


# view_committee

In [15]:
view_committee = grb_committee.rename(columns={'committee_id':'위원회', '선수': '선수평균'}); view_committee

,위원회,결석,청가,출석,출장,합계,결석률,출석률,선수평균
0,4차 산업혁명 특별위원회,19,18,233,0,270,7.04,86.30,1.46
1,가습기살균제 사고 진상규명과 피해구제 및 재발방지 대책마련을 위한 국정조사특별위원회,40,7,153,0,200,20.00,76.50,1.34
2,감사원장(최재형)임명동의에 관한 인사청문특별위원회,5,1,20,0,26,19.23,76.92,1.46
3,과학기술정보방송통신위원회,143,73,1198,0,1414,10.11,84.72,1.85
4,교육문화체육관광위원회,238,80,1911,0,2229,10.68,85.73,1.91
5,국무총리(이낙연) 임명동의에 관한 인사청문특별위원회위원회,0,1,51,0,52,0.00,98.08,1.62
6,국방위원회,137,28,837,0,1002,13.67,83.53,2.48
7,국토교통위원회,151,60,1629,0,1840,8.21,88.53,2.16
8,국회운영위원회,102,55,846,0,1003,10.17,84.35,1.56
9,기획재정위원회,159,125,1440,0,1724,9.22,83.53,2.59


## view_committee_personnel

In [16]:
view_committee_personnel = view_committee.loc[view_committee['위원회'].str.find('인사')!=-1]; view_committee_personnel

,위원회,결석,청가,출석,출장,합계,결석률,출석률,선수평균
2,감사원장(최재형)임명동의에 관한 인사청문특별위원회,5,1,20,0,26,19.23,76.92,1.46
5,국무총리(이낙연) 임명동의에 관한 인사청문특별위원회위원회,0,1,51,0,52,0.00,98.08,1.62
12,대법관(김재형)임명동의에 관한 인사청문특별위원회,7,0,19,0,26,26.92,73.08,1.54
13,"대법관(민유숙,안철상)임명동의에 관한 인사청문특별위원회",3,7,42,0,52,5.77,80.77,1.52
14,"대법관(박정화, 조재연) 임명동의에 관한 인사청문특별위원회위원회",3,0,49,0,52,5.77,94.23,1.38
15,대법원장(김명수)임명동의에관한인사청문특별위원회,1,1,63,0,65,1.54,96.92,1.54
41,헌법재판소장(김이수) 임명동의에 관한 인사청문특별위원회,1,0,38,0,39,2.56,97.44,1.54
42,헌법재판소장(이진성)임명동의에 관한 인사청문특별위원회,3,1,22,0,26,11.54,84.62,1.38


## view_committee_special

In [17]:
view_committee_special = view_committee.loc[(view_committee['위원회'].str.find('인사')==-1)&
                                            (view_committee['위원회'].str.find('특별')!=-1)]; view_committee_special

,위원회,결석,청가,출석,출장,합계,결석률,출석률,선수평균
0,4차 산업혁명 특별위원회,19,18,233,0,270,7.04,86.30,1.46
1,가습기살균제 사고 진상규명과 피해구제 및 재발방지 대책마련을 위한 국정조사특별위원회,40,7,153,0,200,20.00,76.50,1.34
10,남북관계개선특별위원회,17,5,68,0,90,18.89,75.56,1.56
16,미래일자리특별위원회,16,8,100,0,124,12.90,80.65,1.35
17,미세먼지 대책 특별위원회,14,4,69,0,87,16.09,79.31,1.75
18,민생경제특별위원회,20,8,113,0,141,14.18,80.14,1.72
19,박근혜 정부의 최순실 등 민간인에 의한 국정농단 의혹 사건 진상규명을 위한 국정조사...,21,4,282,0,307,6.84,91.86,1.94
22,사법개혁특별위원회,8,5,88,0,101,7.92,87.13,1.75
25,예산결산특별위원회,231,64,1591,0,1886,12.25,84.36,1.80
27,윤리특별위원회,9,2,94,0,105,8.57,89.52,1.58


## view_committee_standing

In [18]:
view_committee_standing = view_committee.loc[(view_committee['위원회'].str.find('인사')==-1)&
                                            (view_committee['위원회'].str.find('특별')==-1)]; view_committee_standing

,위원회,결석,청가,출석,출장,합계,결석률,출석률,선수평균
3,과학기술정보방송통신위원회,143,73,1198,0,1414,10.11,84.72,1.85
4,교육문화체육관광위원회,238,80,1911,0,2229,10.68,85.73,1.91
6,국방위원회,137,28,837,0,1002,13.67,83.53,2.48
7,국토교통위원회,151,60,1629,0,1840,8.21,88.53,2.16
8,국회운영위원회,102,55,846,0,1003,10.17,84.35,1.56
9,기획재정위원회,159,125,1440,0,1724,9.22,83.53,2.59
11,농림축산식품해양수산위원회,161,82,1362,0,1605,10.03,84.86,1.81
20,법제사법위원회,113,41,1425,1,1580,7.15,90.19,2.26
21,보건복지위원회,98,58,1272,2,1430,6.85,88.95,1.86
23,산업통상자원중소벤처기업위원회,209,98,1819,0,2126,9.83,85.56,1.76


# view_member

In [19]:
view_member = grb_member.drop(columns=['seq']); view_member
view_member = view_member.rename(columns={'empNm':'의원'}); view_member

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항,선수
0,강길부,23,5,85,0,113,20.353982,75.221239,무소속,"4선 [ 17대, 18대, 19대, 20대 ]",NaN,4
1,강병원,6,6,112,0,124,4.838710,90.322581,더불어민주당,1선,NaN,1
2,강석진,13,0,82,0,95,13.684211,86.315789,자유한국당,1선,NaN,1
3,강석호,16,0,69,0,85,18.823529,81.176471,자유한국당,"3선 [ 18대, 19대, 20대 ]",NaN,3
4,강창일,17,4,73,0,94,18.085106,77.659574,더불어민주당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN,4
5,강효상,25,13,65,0,103,24.271845,63.106796,자유한국당,1선,NaN,1
6,강훈식,5,3,74,0,82,6.097561,90.243902,더불어민주당,1선,NaN,1
7,경대수,13,0,77,0,90,14.444444,85.555556,자유한국당,"2선 [ 19대, 20대 ]",NaN,2
8,고용진,4,5,65,0,74,5.405405,87.837838,더불어민주당,1선,NaN,1
9,곽대훈,19,0,78,0,97,19.587629,80.412371,자유한국당,1선,NaN,1


In [20]:
view_member.sort_values('청가', ascending=False)

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항,선수
35,김부겸,3,50,30,0,83,3.614458,36.144578,더불어민주당,"4선 [ 16대, 17대, 18대, 20대 ]",NaN,4
86,도종환,2,37,57,0,96,2.083333,59.375000,더불어민주당,"2선 [ 19대, 20대 ]",NaN,2
54,김영춘,2,30,46,0,78,2.564103,58.974359,더불어민주당,"3선 [ 16대, 17대, 20대 ]",NaN,3
52,김영주,2,30,43,0,75,2.666667,57.333333,더불어민주당,"3선 [ 17대, 19대, 20대 ]",NaN,3
285,추미애,22,25,16,0,63,34.920635,25.396825,더불어민주당,"5선 [ 15대, 16대, 18대, 19대, 20대 ]",NaN,5
251,정우택,18,23,70,0,111,16.216216,63.063063,자유한국당,"4선 [ 15대, 16대, 19대, 20대 ]",NaN,4
80,김현미,18,22,43,0,83,21.686747,51.807229,더불어민주당,"3선 [ 17대, 19대, 20대 ]",NaN,3
85,노회찬,11,20,130,0,161,6.832298,80.745342,정의당,"3선 [ 17대, 19대, 20대 ]",NaN,3
252,정운천,7,19,88,0,114,6.140351,77.192982,바른미래당,1선,NaN,1
231,이혜훈,2,18,63,0,83,2.409639,75.903614,바른미래당,"3선 [ 17대, 18대, 20대 ]",NaN,3


# ---------------------------

# Summary

## Count # of conference for each committee

In [21]:
raw_conference_standing = raw_conference.loc[(raw_conference['committee_id'].str.find('인사')==-1)&
                                            (raw_conference['committee_id'].str.find('특별')==-1)]
raw_conference_standing

,committee_id,committee_round,committeename,confer_date,confer_id,rollbook_fetched
0,보건복지위원회,제343회 02차,보건복지위원회,2016-06-21,9002,True
1,환경노동위원회,제343회 01차,환경노동위원회,2016-06-21,9092,True
2,법제사법위원회,제343회 01차,법제사법위원회,2016-06-20,8994,True
3,농림축산식품해양수산위원회,제343회 01차,농림축산식품해양수산위원회,2016-06-20,8995,True
4,산업통상자원중소벤처기업위원회,제343회 01차,산업통상자원중소벤처기업위원회,2016-06-20,8996,True
5,국토교통위원회,제343회 01차,국토교통위원회,2016-06-20,8997,True
6,정보위원회,제343회 01차,정보위원회,2016-06-20,8998,True
7,교육문화체육관광위원회,제343회 01차,교육문화체육관광위원회,2016-06-17,8992,True
8,여성가족위원회,제343회 01차,여성가족위원회,2016-06-17,8993,True
9,보건복지위원회,제343회 01차,보건복지위원회,2016-06-15,8991,True


## Mean of Count

In [22]:
1051 / 16

65.6875

## Group count by committee

In [23]:
# raw_conference_standing
grb_num_of_conference = raw_conference_standing.groupby('committee_id').agg({'committee_id': 'count'}); grb_num_of_conference
grb_num_of_conference = grb_num_of_conference.rename(columns={'committee_id': '회의 횟수'}); grb_num_of_conference
grb_num_of_conference = grb_num_of_conference.reset_index(); grb_num_of_conference
grb_num_of_conference = grb_num_of_conference.rename(columns={'committee_id': '위원회'}); grb_num_of_conference
grb_num_of_conference.sort_values('회의 횟수', ascending=False)

,위원회,회의 횟수
7,법제사법위원회,95
6,농림축산식품해양수산위원회,86
1,교육문화체육관광위원회,82
14,행정안전위원회,82
12,정무위원회,75
5,기획재정위원회,71
9,산업통상자원중소벤처기업위원회,71
11,외교통일위원회,71
15,환경노동위원회,68
8,보건복지위원회,65


## Total Mean of att rate

In [24]:
# rollbook_standing

total = 19351+2056+1001+3  # 22411
# rollbook_standing['attended'].value_counts()
grb_att_rate_of_all = rollbook_standing.groupby('attended').agg({'attended':'count'}); grb_att_rate_of_all
grb_att_rate_of_all = grb_att_rate_of_all.rename(columns={'attended':'횟수'}); grb_att_rate_of_all
# grb_att_rate_of_all = grb_att_rate_of_all.unstack(); grb_att_rate_of_all
grb_att_rate_of_all = grb_att_rate_of_all.reset_index(); grb_att_rate_of_all
grb_att_rate_of_all['비율'] = grb_att_rate_of_all['횟수'] / total * 100; grb_att_rate_of_all

# rollbook_standing.loc[rollbook_standing['attended']=='출장']

,attended,횟수,비율
0,결석,2056,9.174066
1,청가,1001,4.466557
2,출석,19351,86.345991
3,출장,3,0.013386


# 결석률 TOP 20

In [25]:
view_sort_by_abs_rate = view_member.sort_values('결석률', ascending=False).reset_index().drop(columns=['index'])
view_sort_by_abs_rate.head(20)
# 1위 배덕광: 자한당, 엘시티 게이트, 사직
# 18위 김종태: 자한당(새누리), 선거법위반 의원직 상실
# 29위 이철우: 자한당, 경북도지사

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항,선수
0,배덕광,36,1,29,0,66,54.545455,43.939394,자유한국당,2선,"사퇴, 엘시티게이트",2
1,서청원,37,0,35,0,72,51.388889,48.611111,자유한국당,"8선 [ 11대, 13대, 14대, 15대, 16대, 18대, 19대, 20대 ]",NaN,8
2,한선교,39,0,42,0,81,48.148148,51.851852,자유한국당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN,4
3,홍문표,38,0,45,0,83,45.783133,54.216867,자유한국당,"3선 [ 17대, 19대, 20대 ]",NaN,3
4,우상호,43,1,51,0,95,45.263158,53.684211,더불어민주당,"3선 [ 17대, 19대, 20대 ]",NaN,3
5,이종걸,35,1,47,0,83,42.168675,56.626506,더불어민주당,"5선 [ 16대, 17대, 18대, 19대, 20대 ]",NaN,5
6,조원진,32,1,44,0,77,41.558442,57.142857,대한애국당,"3선 [ 18대, 19대, 20대 ]",NaN,3
7,김무성,21,0,30,0,51,41.176471,58.823529,자유한국당,"6선 [ 15대, 16대, 17대, 18대, 19대, 20대 ]",NaN,6
8,추미애,22,25,16,0,63,34.920635,25.396825,더불어민주당,"5선 [ 15대, 16대, 18대, 19대, 20대 ]",NaN,5
9,이해찬,23,0,43,0,66,34.848485,65.151515,더불어민주당,"7선 [ 13대, 14대, 15대, 16대, 17대, 19대, 20대 ]",NaN,7


## 결석 왕중왕

In [26]:
abs_king_standing = set(['배덕광', '서청원', '한선교', '홍문표', '우상호', '이종걸', '조원진', '김무성', '추미애', '이해찬', '홍문종', '김용태',
                     '이장우', '윤한홍', '정진석', '유승민', '김종태', '이군현', '김재원', '김석기',])
abs_king_main = set(['서청원', '김용태', '한선교', '조원진', '유승민', '홍문종', '김무성', '윤한홍', '이우현', '이해찬', '김석기', '홍문표', '김광림',
                 '김성찬', '이철우', '김정훈', '박명재', '이장우', '최경환', '김영우',])
abs_king_intersection = list(abs_king_standing.intersection(abs_king_main)); abs_king_intersection
df_abs_king_intersection = pd.DataFrame(columns=['의원'], data=abs_king_intersection); df_abs_king_intersection
df_abs_king_intersection = df_abs_king_intersection.merge(grb_member[['empNm','선수','정당']], left_on='의원', right_on='empNm', how='left'); df_abs_king_intersection
df_abs_king_intersection = df_abs_king_intersection.sort_values(['정당','의원'], ascending=[False, True]); df_abs_king_intersection
df_abs_king_intersection = df_abs_king_intersection.drop(columns=['empNm']); df_abs_king_intersection

df_abs_king_intersection = df_abs_king_intersection.merge(view_sort_by_abs_rate[['의원', '결석률', '결석', '합계']]); df_abs_king_intersection
abs_king_list = {
    '김무성': 23,
    '김석기': 16,
    '김용태': 29,
    '서청원': 46,
    '윤한홍': 19,
    '이장우': 14,
    '한선교': 27,
    '홍문종': 23,
    '홍문표': 16,
    '유승민': 24,
    '이해찬': 18,
    '조원진': 24,
}
df_abs_king_intersection['본회의결석횟수'] = df_abs_king_intersection['의원'].apply(lambda x: abs_king_list[x]); df_abs_king_intersection
df_abs_king_intersection['본회의합계횟수'] = 84
df_abs_king_intersection['본회의결석률'] = round(df_abs_king_intersection['본회의결석횟수'] / 84 * 100, 2); df_abs_king_intersection
# df_abs_king_intersection['종합 결석률'] = round((df_abs_king_intersection['결석률'] + df_abs_king_intersection['본회의결석률'])/2, 2); df_abs_king_intersection
df_abs_king_intersection['종합 결석률'] = round((df_abs_king_intersection['결석'] + df_abs_king_intersection['본회의결석횟수'])/
                                             (df_abs_king_intersection['합계'] + df_abs_king_intersection['본회의합계횟수']) * 100, 2); df_abs_king_intersection
df_abs_king_intersection['결석률'] = round(df_abs_king_intersection['결석률'], 2); df_abs_king_intersection
df_abs_king_intersection = df_abs_king_intersection.sort_values('종합 결석률', ascending=False); df_abs_king_intersection
df_abs_king_intersection = df_abs_king_intersection[['의원', '정당', '종합 결석률', '결석률', '본회의결석률', '선수']]; df_abs_king_intersection
df_abs_king_intersection = df_abs_king_intersection.rename(columns={'결석률': '상임위결석률', '선수': '당선횟수'}); df_abs_king_intersection

,의원,정당,종합 결석률,상임위결석률,본회의결석률,당선횟수
3,서청원,자유한국당,53.21,51.39,54.76,8
6,한선교,자유한국당,40.00,48.15,32.14,4
11,조원진,대한애국당,34.78,41.56,28.57,3
2,김용태,자유한국당,34.36,34.18,34.52,3
0,김무성,자유한국당,32.59,41.18,27.38,6
8,홍문표,자유한국당,32.34,45.78,19.05,3
7,홍문종,자유한국당,30.15,34.62,27.38,4
9,유승민,바른미래당,29.53,30.77,28.57,4
10,이해찬,더불어민주당,27.33,34.85,21.43,7
4,윤한홍,자유한국당,26.88,31.58,22.62,1


In [27]:
df_abs_king_intersection['의원']

3     서청원
6     한선교
11    조원진
2     김용태
0     김무성
8     홍문표
7     홍문종
9     유승민
10    이해찬
4     윤한홍
5     이장우
1     김석기
Name: 의원, dtype: object

## 결석률TOP20 정당분포

In [28]:
view_sort_by_abs_rate.head(20).groupby('정당').agg({'정당':'count'})

,정당
정당,
대한애국당,1
더불어민주당,4
바른미래당,1
자유한국당,14


# 출석률 TOP 20

In [29]:
view_sort_by_att_rate = view_member.sort_values(['출석률', '합계'], ascending=False).reset_index().drop(columns=['index'])
view_sort_by_att_rate.head(20)
# 문미옥: 더민주, 대통령비서실 과학기술보좌관
# 양승조: 더민주, 충남도지사후보

# 박남춘: 더민주, 인천시장후보

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항,선수
0,전재수,0,0,104,0,104,0.000000,100.000000,더불어민주당,1선,NaN,1
1,박광온,0,0,97,0,97,0.000000,100.000000,더불어민주당,"2선 [ 19대, 20대 ]",NaN,2
2,유동수,0,0,94,0,94,0.000000,100.000000,더불어민주당,1선,NaN,1
3,김민기,0,0,84,0,84,0.000000,100.000000,더불어민주당,"2선 [ 19대, 20대 ]",NaN,2
4,유성엽,0,0,81,0,81,0.000000,100.000000,민주평화당,"3선 [ 18대, 19대, 20대 ]",NaN,3
5,박남춘,0,0,70,0,70,0.000000,100.000000,더불어민주당,2선,"사퇴, 인천시장후보",2
6,양승조,0,0,65,0,65,0.000000,100.000000,더불어민주당,4선,"사퇴, 충남도지사후보",4
7,이원욱,0,0,62,0,62,0.000000,100.000000,더불어민주당,"2선 [ 19대, 20대 ]",NaN,2
8,조정식,0,0,60,0,60,0.000000,100.000000,더불어민주당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN,4
9,문미옥,0,0,50,0,50,0.000000,100.000000,더불어민주당,1선,"사퇴, 대통령비서실 과학기술보좌관",1


## 출석 왕중왕

In [30]:
att_king_standing = set(['전재수', '박광온', '유동수', '김민기', '유성엽', '박남춘', '양승조', '이원욱', '조정식', '문미옥', '이수혁', '진선미',
                         '정성호', '최인호', '기동민', '최도자', '유은혜', '김영진', '김영호', '박주민',
])
att_king_main = set(['박홍근', '박찬대', '김병관', '김병욱', '유동수', '박주현', '박주민', '박정', '김태년', '윤관석', '정양석', '김영진', '박남춘',
                     '박광온', '김정우', '전재수', '문희상', '백혜련', '김상희', '송기헌',
])
att_king_intersection = list(att_king_standing.intersection(att_king_main)); att_king_intersection
df_att_king_intersection = pd.DataFrame(columns=['의원'], data=att_king_intersection); df_att_king_intersection
df_att_king_intersection = df_att_king_intersection.merge(grb_member[['empNm','선수','정당', '출석률', '합계', '출석']], left_on='의원', right_on='empNm',
                                                          how='left'); df_att_king_intersection
# df_att_king_intersection = df_att_king_intersection.sort_values(['의원'], ascending=[True]); df_att_king_intersection
df_att_king_intersection = df_att_king_intersection.drop(columns=['empNm']); df_att_king_intersection
df_att_king_intersection['종합 출석률'] = round((df_att_king_intersection['출석']+84) / (df_att_king_intersection['합계']+84) * 100 ,2); df_att_king_intersection
df_att_king_intersection['본회의 출석률'] = 100.00
df_att_king_intersection['출석률'] = round(df_att_king_intersection['출석률'], 2)
df_att_king_intersection = df_att_king_intersection.sort_values(['종합 출석률', '합계'], ascending=False); df_att_king_intersection
df_att_king_intersection = df_att_king_intersection[['의원', '정당', '종합 출석률', '출석률', '본회의 출석률', '선수']]; df_att_king_intersection
df_att_king_intersection = df_att_king_intersection.rename(columns={'출석률': '상임위 출석률'}); df_att_king_intersection

,의원,정당,종합 출석률,상임위 출석률,본회의 출석률,선수
3,전재수,더불어민주당,100.00,100.00,100.0,1
4,박광온,더불어민주당,100.00,100.00,100.0,2
0,유동수,더불어민주당,100.00,100.00,100.0,1
5,박남춘,더불어민주당,100.00,100.00,100.0,2
1,김영진,더불어민주당,99.39,98.73,100.0,1
2,박주민,더불어민주당,99.14,98.66,100.0,1


## 출석률TOP20 정당분포

In [31]:
view_sort_by_att_rate.head(20).groupby('정당').agg({'정당':'count'})

,정당
정당,
더불어민주당,18
민주평화당,1
바른미래당,1


# 정당별 출석률

In [32]:
# 결석	청가	출석	출장	합계
view_party_att = view_member[['정당','청가','출장','결석','출석','합계']].groupby('정당').agg('sum'); view_party_att
view_party_att['결석률'] = view_party_att['결석'] / view_party_att['합계'] * 100; view_party_att
view_party_att['출석률'] = view_party_att['출석'] / view_party_att['합계'] * 100; view_party_att
view_party_att = view_party_att.reset_index(); view_party_att

,정당,청가,출장,결석,출석,합계,결석률,출석률
0,대한애국당,1,0,32,44,77,41.558442,57.142857
1,더불어민주당,564,1,637,10200,11402,5.586739,89.457990
2,무소속,23,0,60,294,377,15.915119,77.984085
3,민주평화당,75,0,81,1288,1444,5.609418,89.196676
4,민중당,8,0,20,118,146,13.698630,80.821918
5,바른미래당,158,0,223,2660,3041,7.333114,87.471227
6,자유한국당,387,2,1678,9024,11091,15.129384,81.363268
7,정의당,33,0,55,542,630,8.730159,86.031746


## 정당 결석률 순위

In [33]:
view_party_sort_abs = view_party_att.loc[view_party_att['출석']>500].sort_values('결석률', ascending=False); view_party_sort_abs

,정당,청가,출장,결석,출석,합계,결석률,출석률
6,자유한국당,387,2,1678,9024,11091,15.129384,81.363268
7,정의당,33,0,55,542,630,8.730159,86.031746
5,바른미래당,158,0,223,2660,3041,7.333114,87.471227
3,민주평화당,75,0,81,1288,1444,5.609418,89.196676
1,더불어민주당,564,1,637,10200,11402,5.586739,89.457990


## 정당 출석률 순위

In [34]:
view_party_sort_att = view_party_att.loc[view_party_att['출석']>500].sort_values('출석률', ascending=False); view_party_sort_att

,정당,청가,출장,결석,출석,합계,결석률,출석률
1,더불어민주당,564,1,637,10200,11402,5.586739,89.457990
3,민주평화당,75,0,81,1288,1444,5.609418,89.196676
5,바른미래당,158,0,223,2660,3041,7.333114,87.471227
7,정의당,33,0,55,542,630,8.730159,86.031746
6,자유한국당,387,2,1678,9024,11091,15.129384,81.363268


# 상임위 출석률

In [35]:
view_committee_standing.sort_values('출석률', ascending=False)

,위원회,결석,청가,출석,출장,합계,결석률,출석률,선수평균
37,행정안전위원회,53,65,1386,0,1504,3.52,92.15,1.87
24,여성가족위원회,32,15,480,0,527,6.07,91.08,1.15
20,법제사법위원회,113,41,1425,1,1580,7.15,90.19,2.26
21,보건복지위원회,98,58,1272,2,1430,6.85,88.95,1.86
7,국토교통위원회,151,60,1629,0,1840,8.21,88.53,2.16
30,정무위원회,118,105,1577,0,1800,6.56,87.61,1.79
4,교육문화체육관광위원회,238,80,1911,0,2229,10.68,85.73,1.91
23,산업통상자원중소벤처기업위원회,209,98,1819,0,2126,9.83,85.56,1.76
11,농림축산식품해양수산위원회,161,82,1362,0,1605,10.03,84.86,1.81
3,과학기술정보방송통신위원회,143,73,1198,0,1414,10.11,84.72,1.85


# ---------------------------

# Visualization

## 상임위별 출석률 & 결석률

In [36]:
view_committee_standing.sort_values('출석률', ascending=True)

,위원회,결석,청가,출석,출장,합계,결석률,출석률,선수평균
26,외교통일위원회,137,86,917,0,1140,12.02,80.44,4.01
6,국방위원회,137,28,837,0,1002,13.67,83.53,2.48
9,기획재정위원회,159,125,1440,0,1724,9.22,83.53,2.59
31,정보위원회,58,8,340,0,406,14.29,83.74,2.89
8,국회운영위원회,102,55,846,0,1003,10.17,84.35,1.56
43,환경노동위원회,147,22,912,0,1081,13.60,84.37,1.33
3,과학기술정보방송통신위원회,143,73,1198,0,1414,10.11,84.72,1.85
11,농림축산식품해양수산위원회,161,82,1362,0,1605,10.03,84.86,1.81
23,산업통상자원중소벤처기업위원회,209,98,1819,0,2126,9.83,85.56,1.76
4,교육문화체육관광위원회,238,80,1911,0,2229,10.68,85.73,1.91


In [82]:
# view_committee_standing
chart_committee_standing = view_committee_standing.sort_values('출석률', ascending=False)
chart_committee_standing = chart_committee_standing.rename(columns={'선수평균':'평균당선횟수'})

chart_committee_standing['위원회'] = chart_committee_standing['위원회'].str.replace('위원회', '')
# header: '위원회', '결석률', '출석률', '선수평균'
header, cells = get_header_and_cells_from_df(chart_committee_standing[['위원회', '출석률', '결석률', '평균당선횟수']])

columnwidth_standing_ratio = [17, 6, 6, 8]
columnwidth_standing_sum = sum(columnwidth_standing_ratio)
columnwidth_standing = [x / columnwidth_standing_sum * 600 for x in columnwidth_standing_ratio]
print(sum(columnwidth_standing))
trace = go.Table(
    columnwidth=columnwidth_standing,
    header=dict(
        values=header,
        fill=dict(color='#e8f4ee'),
        font=dict(size=14,)
    ),
    cells=dict(
        values=cells,
        height=25,
        fill=dict(color=['#fff', '#ffffcc', '#fff', ['#fff' if cell != 4.01 else '#ffffcc' for cell in cells[-1]]]),
        font=dict(size=14,)
    ),
)
layout = go.Layout(
    title='상임위별 출석률 & 결석률<span style="font-size:0.7em"> (단위: %)</span>',
    width=600,
#     width=sum(columnwidth_standing),
    height=500,
    margin=dict(t=40,r=20,b=20,l=20),
)
data = [trace]
fig_0_att_rate_for_standing = dict(data=data, layout=layout)
off.iplot(fig_0_att_rate_for_standing)

600.0


## 의원 결석률 & 출석률

### 결석률 TOP 20

In [93]:
chart_abs_20 = view_sort_by_abs_rate.head(20); chart_abs_20
chart_abs_20 = chart_abs_20.sort_values('결석률', ascending=True); chart_abs_20
chart_abs_20 = chart_abs_20[['의원', '결석률', '출석률', '정당', '선수', '특이사항']]; chart_abs_20
chart_abs_20['결석률'] = round(chart_abs_20['결석률'], 2); chart_abs_20
chart_abs_20['출석률'] = round(chart_abs_20['출석률'], 2); chart_abs_20
chart_abs_20 = chart_abs_20.fillna(''); chart_abs_20
chart_abs_20 = chart_abs_20.rename(columns={'선수': '당선횟수'}); chart_abs_20

hover0 = '당선횟수: ' + chart_abs_20['당선횟수'].astype(str)
hover1 = '출석률: ' + chart_abs_20['출석률'].astype(str)
hover2 = '특이사항: ' + chart_abs_20['특이사항'].astype(str)
hovertext = get_hovertext_list(chart_abs_20['정당'], hover0, hover1, hover2)

party_color = get_party_color_dict(chart_abs_20['정당'])
trace = dict(
    type='bar',
    x=chart_abs_20['결석률'],
    y=chart_abs_20['의원'],
    orientation='h',
    marker=party_color,
    hoverinfo='text',
    hovertext=hovertext,
)
annotations = [make_annotation(x, y) for x, y in zip(chart_abs_20['결석률'], chart_abs_20['의원'])]
legend_text = get_legend_text_party(chart_abs_20['정당'], y_start=0.162, x=0.98)
annotations.extend(legend_text)
shapes = get_legend_rect_party(chart_abs_20['정당'], rect_width=0.06, y_start=0.2, x0=1)
layout = dict(
    title='상임위 결석률 TOP 20',
    height=530,
    width=600,
    margin=dict(
        pad=2.5,
        t=50,r=50,b=50,l=50,
    ),
    annotations=annotations,
    shapes=shapes,
    xaxis=dict(
        title='결석률<span style="font-size:0.7em"> (단위: %)</span>',
        showticklabels=False,
        zeroline=False,
        fixedrange=True,
        showgrid=False,
    ),
    yaxis=dict(
        fixedrange=True,
    ),
)
data = [trace]
fig_1_abs_rate = dict(data=data, layout=layout)
off.iplot(fig_1_abs_rate)

### 결석 왕중왕

In [84]:
party_color = {
    '더불어민주당': '#004EA2',
    '자유한국당': '#C8161E',
    '바른미래당': '#20B2AA',
    '민주평화당': '#40B02A',
    '정의당': '#FFCC00',
    '대한애국당': '#080B54',
    '민중당': '#F47920',
    '무소속': '#a6a6a6',
}

df_abs_king_intersection = df_abs_king_intersection.rename(columns={
    '종합 결석률': '종합<br>결석률',
    '상임위결석률': '상임위<br>결석률',
    '본회의결석률': '본회의<br>결석률',
    '당선횟수': '당선<br>횟수',
})
# # header: '의원', '정당', '종합 결석률', '상임위결석률', '본회의결석률', '당선횟수'
header, cells = get_header_and_cells_from_df(df_abs_king_intersection)
color_party = [party_color[x] for x in df_abs_king_intersection['정당']]
color_red = cl.scales['9']['seq']['YlGn'][::-1]
color_grey0 = list(cl.scales['9']['seq']['Greys'])
color_grey0[:7] = ['white']*6
# columnwidth_abs = [60, 90, 60, 60, 60, 40]

columnwidth_abs_ratio = [6, 9, 6, 6, 6, 4]
columnwidth_abs_sum = sum(columnwidth_abs_ratio)
columnwidth_abs = [x / columnwidth_abs_sum * 600 for x in columnwidth_abs_ratio]

print(sum(columnwidth_abs))
trace = go.Table(
    columnwidth=600,
    header=dict(
        values=header,
        line=dict(color = '#fff'),
        fill=dict(color='#e8f4ee'),
        font=dict(size=14,),
    ),
    cells=dict(
        values=cells,
        height=25,
        line = dict(color = ['#F5F5F5', color_party, color_red, '#F5F5F5']),
        fill = dict(color = ['#F5F5F5', color_party, color_red, '#F5F5F5']),
        font = dict(color = ['black', 'white', color_grey0, 'black'], size = 14)
    ),
)
layout = go.Layout(
    title='20대 국회 결석 왕중왕<span style="font-size:0.7em"> (단위: %)</span>',
    width=600,
    height=450,
    margin=dict(t=70,r=20,b=20,l=20),
)
data = [trace]
fig_1_1_abs_king = dict(data=data, layout=layout)
off.iplot(fig_1_1_abs_king)

600.0000000000001


### 출석률 TOP 20

In [94]:
def make_annotation(x, y):
    return go.Annotation(
        text=str(x) if x != 100.0 else '개근',
        showarrow=False, # annotation w/o arrows, default is True
        x=x,               # set x position
        xref='x',          # position text horizontally with x-coords
        xanchor='left',  # x position corresp. to center of text
        yref='y',            # set y position
        yanchor='auto',       # position text vertically with y-coords
        y=y,                 # y position corresp. to top of text
        font=go.Font(
            color='#262626',  # set font color
            size=13           #   and size
        )
    )

chart_att_20 = view_sort_by_att_rate.head(20); chart_att_20
chart_att_20 = chart_att_20.sort_values(['출석률', '합계'], ascending=True); chart_att_20
chart_att_20 = chart_att_20[['의원', '결석률', '출석률', '정당', '선수', '특이사항']]; chart_att_20
chart_att_20['결석률'] = round(chart_att_20['결석률'], 2); chart_att_20
chart_att_20['출석률'] = round(chart_att_20['출석률'], 2); chart_att_20
chart_att_20 = chart_att_20.fillna(''); chart_att_20
chart_att_20 = chart_att_20.rename(columns={'선수': '당선횟수'}); chart_att_20
# chart_att_20['출석률'] = chart_att_20['출석률'].apply(lambda x: '개근' if x == 100.0 else x)

hover0 = '당선횟수: ' + chart_att_20['당선횟수'].astype(str)
hover1 = '출석률: ' + chart_att_20['출석률'].astype(str)
hover2 = '특이사항: ' + chart_att_20['특이사항'].astype(str)
hovertext = get_hovertext_list(chart_att_20['정당'], hover0, hover1, hover2)

party_color = get_party_color_dict(chart_att_20['정당'])
trace = dict(
    type='bar',
    x=chart_att_20['출석률'],
    y=chart_att_20['의원'],
    orientation='h',
    marker=party_color,
    hoverinfo='text',
    hovertext=hovertext,
)
als = ['개근' if x == 100.0 else x for x in chart_att_20['출석률']]
# print(als)
annotations = [make_annotation(x, y) for x, y in zip(chart_att_20['출석률'], chart_att_20['의원'])]
legend_text = get_legend_text_party(chart_att_20['정당'], y_start=-0.242, x=0.98)
annotations.extend(legend_text)
shapes = get_legend_rect_party(chart_att_20['정당'], rect_width=0.06, y_start=-0.2, x0=1)
layout = dict(
    title='상임위 출석률 TOP 20',
    height=530,
    width=600,
    margin=dict(
        pad=2.5,
        t=50,r=50,b=150,l=50,
    ),
    annotations=annotations,
    shapes=shapes,
    xaxis=dict(
        title='출석률<span style="font-size:0.7em"> (단위: %)</span>',
        showticklabels=False,
        zeroline=False,
        fixedrange=True,
        showgrid=False,
    ),
    yaxis=dict(
        fixedrange=True,
    ),
)
data = [trace]
fig_2_att_rate = dict(data=data, layout=layout)
off.iplot(fig_2_att_rate)

### 출석 왕중왕

In [85]:
party_color = {
    '더불어민주당': '#004EA2',
    '자유한국당': '#C8161E',
    '바른미래당': '#20B2AA',
    '민주평화당': '#40B02A',
    '정의당': '#FFCC00',
    '대한애국당': '#080B54',
    '민중당': '#F47920',
    '무소속': '#a6a6a6',
}

df_att_king_intersection = df_att_king_intersection.rename(columns={
    '종합 출석률': '종합<br>출석률',
    '상임위 출석률': '상임위<br>출석률',
    '본회의 출석률': '본회의<br>출석률',
    '선수': '당선<br>횟수',
})
# # header: '의원', '정당', '종합 출석률', '상임위 출석률', '본회위 출석률', '당선횟수'
header, cells = get_header_and_cells_from_df(df_att_king_intersection)
color_party = [party_color[x] for x in df_att_king_intersection['정당']]
color_green = cl.scales['9']['seq']['YlGn'][::-1]
color_grey0 = ['white']*6
columnwidth_att = [60, 90, 60, 60, 60, 40]

columnwidth_att_ratio = [6, 9, 6, 6, 6, 4]
columnwidth_att_sum = sum(columnwidth_att_ratio)
columnwidth_att = [x / columnwidth_att_sum * 600 for x in columnwidth_att_ratio]

print(sum(columnwidth_att))
trace = go.Table(
    columnwidth=600,
    header=dict(
        values=header,
        line=dict(color='#fff'),
        fill=dict(color='#e8f4ee'),
        font=dict(size=14,),
    ),
    cells=dict(
        values=cells,
        height=25,
        line = dict(color = ['#F5F5F5', color_party, color_green, '#F5F5F5']),
        fill = dict(color = ['#F5F5F5', color_party, color_green, '#F5F5F5']),
        font = dict(color = ['black', 'white', color_grey0, 'black'], size = 14)
    ),
)
layout = go.Layout(
    title='20대 국회 출석 왕중왕<span style="font-size:0.7em"> (단위: %)</span>',
    width=600,
    height=300,
    margin=dict(t=70,r=20,b=20,l=20),
)
data = [trace]
fig_1_2_att_king = dict(data=data, layout=layout)
off.iplot(fig_1_2_att_king)

600.0000000000001


## 정당 결석률 & 출석률: 상호보완적인 결석률과 출석률 보여줄 수 있는 그래프?

In [86]:
chart_party_att = view_party_att.loc[view_party_att['출석']>500].sort_values('결석률', ascending=False); chart_party_att
chart_party_att = chart_party_att[['정당', '결석률', '출석률']]; chart_party_att
chart_party_att['결석률'] = round(chart_party_att['결석률'], 2); chart_party_att
chart_party_att['출석률'] = round(chart_party_att['출석률'], 2); chart_party_att

# # header: '정당', '결석률', '출석률'
header, cells = get_header_and_cells_from_df(chart_party_att)
# print(cells[1])
color_red = cl.scales['5']['seq']['Reds'][::-1]
# print(color_red)
color_blue = cl.scales['5']['seq']['Blues']
color_grey0 = list(cl.scales['5']['seq']['Greys'])
color_grey0[:3] = ['white']*3
color_grey1 = list(cl.scales['5']['seq']['Greys'][::-1])
color_grey1[2:] = ['white']*3
# columnwidth_party = [110, 65, 65]

columnwidth_party_ratio = [110, 65, 65]
columnwidth_party_sum = sum(columnwidth_party_ratio)
columnwidth_party = [x / columnwidth_party_sum * 600 for x in columnwidth_party_ratio]


print(sum(columnwidth_party))
trace = go.Table(
    columnwidth=600,
    header=dict(
        values=header,
        line=dict(color = '#fff'),
        fill=dict(color='#e8f4ee'),
        font=dict(size=14,),
    ),
    cells=dict(
        values=cells,
        height=25,
        line = dict(color = ['#F5F5F5', color_red, color_blue]),
        fill = dict(color = ['#F5F5F5', color_red, color_blue]),
        font = dict(color = ['black', color_grey0, color_grey1], size = 14)
    ),
)
layout = go.Layout(
    title='정당별 상임위회의<br>결석률 & 출석률<span style="font-size:0.7em"> (단위: %)</span>',
    width=600,
    height=250,
    margin=dict(t=70,r=20,b=20,l=20),
)
data = [trace]
fig_3_party_att = dict(data=data, layout=layout)
off.iplot(fig_3_party_att)

600.0


# ---------------------------

# iplot

In [96]:
# # fig_0_standing
# py.iplot(fig_0_att_rate_for_standing, filename='fig_0_standing')

# # fig_1_abs
# py.iplot(fig_1_abs_rate, filename='fig_1_abs_rate')

# fig_1_1_abs_king
# py.iplot(fig_1_1_abs_king, filename='fig_1_1_abs_king')

# fig_1_2_att_king
# py.iplot(fig_1_2_att_king, filename='fig_1_2_att_king')

# # fig_2_att
# py.iplot(fig_2_att_rate, filename='fig_2_att_rate')

# # fig_3_party
# py.iplot(fig_3_party_att, filename='fig_3_party_att')


# fig_0_standing
# off.iplot(fig_0_att_rate_for_standing, filename='fig_0_standing', image='jpeg',
#           image_width=600, image_height=500,)

# # fig_1_abs
# off.iplot(fig_1_abs_rate, filename='fig_1_abs_rate', image='jpeg',
#           image_width=600, image_height=530,)

# fig_1_1_abs_king
# off.iplot(fig_1_1_abs_king, filename='fig_1_1_abs_king', image='jpeg',
#           image_width=600, image_height=450,)

# fig_1_2_att_king
# off.iplot(fig_1_2_att_king, filename='fig_1_2_att_king', image='jpeg',
#           image_width=600, image_height=300,)

# # fig_2_att
# off.iplot(fig_2_att_rate, filename='fig_2_att_rate', image='jpeg',
#           image_width=600, image_height=530,)

# # fig_3_party
# off.iplot(fig_3_party_att, filename='fig_3_party_att', image='jpeg',
#           image_width=600, image_height=250,)

# ---------------------------

# Save

## view_member

In [44]:
# save_df_to_tsv(view_member, '../data/20대국회상임위출석부.tsv')

## view_sort_by_abs_rate

In [45]:
# save_df_to_tsv(view_sort_by_abs_rate.head(30), '../data/의원결석률순위30.tsv')

## view_sort_by_att_rate

In [46]:
# save_df_to_tsv(view_sort_by_att_rate.head(30), '../data/의원출석률순위30.tsv')

## view_party_sort_abs

In [47]:
# save_df_to_tsv(view_party_sort_abs, '../data/정당결석률순위.tsv')

## view_party_sort_att

In [48]:
# save_df_to_tsv(view_party_sort_att, '../data/정당출석률순위.tsv')